**Downloading Data and Loads it**

In [1]:
# @title Default title text
# download the test set. replace the link with the train set download link if you want to
!wget -O test.zip --no-check-certificate https://huggingface.co/datasets/ccdv/arxiv-summarization/resolve/main/test.zip?download=true

--2024-04-03 14:00:54--  https://huggingface.co/datasets/ccdv/arxiv-summarization/resolve/main/test.zip?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.103, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/ccdv/arxiv-summarization/8ee7cf45fde92768515e2f3170ecb1cf9bdae60169f2b4d4f9b60f1f628e862c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27test.zip%3B+filename%3D%22test.zip%22%3B&response-content-type=application%2Fzip&Expires=1712412055&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMjQxMjA1NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9jY2R2L2FyeGl2LXN1bW1hcml6YXRpb24vOGVlN2NmNDVmZGU5Mjc2ODUxNWUyZjMxNzBlY2IxY2Y5YmRhZTYwMTY5ZjJiNGQ0ZjliNjBmMWY2MjhlODYyYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVud

In [2]:
# extract the downloaded file
import zipfile
with zipfile.ZipFile("test.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [3]:
!head -1 test.txt

{"article_id": "1009.3123", "article_text": ["for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively .", "many investigators studied the short - term periodicities of the various indices of solar activity .", "several periodicities were detected , but the periodicities about 155 days and from the interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .", "first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .", "this periodicity was confirmed for other solar flares data and for the same time period @xcite .", "it was also found in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar flares data during solar cycles 22 @xcite .", "_    several autors confirmed above results for the daily sunspot area data . @xcite studied the sunspot data from 18741984 .", "s

In [4]:
# load papers in json format from each line of the extracted file
papers = []
import json
article_ids = []
f = 1
with open('test.txt', 'r') as file:
    for line in file:
        try:
            # Parse the JSON data from each line
            paper_data = json.loads(line)
            if f == 1:
              print("paper_data: ", paper_data)
              f += 1
            # Extract the arXiv ID and append to the list
            if 'article_id' in paper_data:
                article_ids.append(paper_data['article_id'])
            papers.append(paper_data)
        except json.JSONDecodeError as e:
            # Output an error message if a line is not valid JSON
            print(f"Error parsing JSON for line: {line}")
            print(str(e))

# Now `article_ids` contains all the article IDs from the file
# print(article_ids)


paper_data:  {'article_id': '1009.3123', 'article_text': ['for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively .', 'many investigators studied the short - term periodicities of the various indices of solar activity .', 'several periodicities were detected , but the periodicities about 155 days and from the interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .', 'first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .', 'this periodicity was confirmed for other solar flares data and for the same time period @xcite .', 'it was also found in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar flares data during solar cycles 22 @xcite .', '_    several autors confirmed above results for the daily sunspot area data . @xcite studied the sunspot data from 18

Papers - literally has everything. all the article id's and texts and all that stuff.
article id's - only contain article id's

In [ ]:
# counting the number of astro physics papers in the dataset
count = 0
for i in article_ids:
  if i.startswith("astro-ph"):
    count += 1
print(count)

1046


In [ ]:
# total article ids
len(article_ids)

6440

In [ ]:
# print the beautified paper which is in JSON format
json_formatted_str = json.dumps(papers[0], indent=2)
print(json_formatted_str)

## Skip to LSA Summarization

In [5]:
import nltk
import numpy as np
import re

nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #print("tokens: ", tokens)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    #print("filtered tokens: ", filtered_tokens)
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    #print("doc after joining: ", doc)
    return doc

normalize_corpus = np.vectorize(normalize_document)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [7]:
from scipy.sparse.linalg import svds

def low_rank_svd(matrix, singular_count=2):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

In [8]:
# Get top n sentences. Default value of n is 4.
def get_sentences_top(sentences, num_sentences=4):
    norm_sentences = normalize_corpus(sentences)

    norm_sentences = list(filter(None, norm_sentences))
    #print("norm sentences: ", norm_sentences)

    tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)

    if len(norm_sentences) == 0:
        return np.array([])

    dt_matrix = tv.fit_transform(norm_sentences)
    #print("dt matrix: ", dt_matrix)
    dt_matrix = dt_matrix.toarray()
    #print("dt matrix after converting it to array: ", dt_matrix)
    vocab = tv.get_feature_names_out()
    #print("Vocab: ", vocab)
    td_matrix = dt_matrix.T
    #print(td_matrix.shape)
    pd.DataFrame(np.round(td_matrix, 2), index=vocab).head(10)
    num_sentences = min(num_sentences, len(sentences))
    #print("num_sentences", num_sentences)
    num_topics = 3
    if(len(norm_sentences) <= num_topics):
        return np.array(sentences)

    u, s, vt = low_rank_svd(td_matrix, singular_count=num_topics)
    #print(u.shape, s.shape, vt.shape)
    term_topic_mat, singular_values, topic_document_mat = u, s, vt

    sv_threshold = 0.5
    min_sigma_value = max(singular_values) * sv_threshold
    singular_values[singular_values < min_sigma_value] = 0
    salience_scores = np.sqrt(np.dot(np.square(singular_values),
                                 np.square(topic_document_mat)))
    top_sentence_indices = (-salience_scores).argsort()[:num_sentences]
    top_sentence_indices.sort()
    return np.array(sentences)[top_sentence_indices]

TF-IDF gives dt matrix. It's document-term matrix. Every row represents a doc and every column represents words. It gives how relevant each word is, and they can be used as embeddings as prof explained.

In [ ]:
# Sample paper from the dataset
paper = papers[1]
sections = paper["sections"]
sentences = sections[0]
get_sentences_top(sentences, 5)


array(['it is believed that the direct detection of gravitational waves ( gws ) will bring the era of gravitational wave astronomy .',
       'the first direct detection of the gravitational waves might be achieved by ptas .',
       'we also discuss a method to separate the intensity ( @xmath3 mode ) and circular polarization ( @xmath2 mode ) of the sgwb .',
       'the basic framework is essentially a combination of the formalism of @xcite , and the polarization decomposition formula of the sgwb derived in @xcite . in section [ sec : the generalized overlap reduction function for circular polarization ] , we calculate the generalized orfs for the @xmath2 mode .',
       'the results for @xmath3 mode are consistent with the previous work  @xcite . in section [ sec : separation method ] , we give a method for separation between the @xmath3 mode and @xmath2 mode of the sgwb .'],
      dtype='<U349')

It parallel processes. It acts as if it's executing and processing for each statement in the section seperately. That's why it's giving print statement like that. Escpecially that we're using a vectorize function. 'np.vectorize' enables you to apply a regular Python function to NumPy arrays in a vectorized manner, making it convenient for performing element-wise operations on arrays using custom functions. Look at the example below.

In [ ]:
# Define a regular Python function that operates on a scalar
def my_function(x):
  y = x ** 2 + 3 * x + 2
  print("y: ", y)
  return y

# Vectorize the function using np.vectorize
vectorized_function = np.vectorize(my_function)

# Create a NumPy array
arr = np.array([1, 2, 3, 4, 5])

# Apply the vectorized function to the array
result = vectorized_function(arr)

print("result: ", result)


In [ ]:
# Sample sectionwise summaries for a paper
paper = papers[1]
sections = paper["sections"]
extractive_summaries = []
for section in sections:
  extractive_summaries.append(get_sentences_top(section, 5))

In [9]:
# Define a method to perform extractive summarization)
def extractive(document_sections, num_sentences=4):
    overall_summary = ""
    for section in document_sections:
        summary = get_sentences_top(section, num_sentences)
        overall_summary += " ".join(summary) + " "
    return overall_summary

In [10]:
!pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=be4a54e96ab0eb73f0d37d767833df90e170d9f756f291ab3c32718033e19a3f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [11]:
from datasets import __version__ as ver
print(ver)

2.18.0


In [12]:
# load the data as huggingface dataset object from papers list
from datasets import Dataset, load_dataset
papers_ds = Dataset.from_pandas(pd.DataFrame(data=papers))
papers_ds

Dataset({
    features: ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'],
    num_rows: 6440
})

In [13]:
papers_split = papers_ds.train_test_split(test_size=0.2)
rpapers = papers_split
# access paper like this papers_split["train"][0]

In [16]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPTNeoForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [17]:
model_name = "EleutherAI/gpt-neo-125M"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [22]:
prefix = "summarize: "

def preprocess_function(batched_examples):
    inputs = [prefix + extractive(sections, num_sentences=4) for sections in batched_examples["sections"]]

    input_tokens = tokenizer(inputs, truncation=True, padding=True, max_length=512, return_tensors="pt")
    check_outputs = [abstract_combiner(abstract) for abstract in batched_examples["abstract_text"]]
    label_tokens = tokenizer(check_outputs, truncation=True, padding=True, max_length=400, return_tensors="pt")



    # Remove labels from input_ids to avoid training on them
    input_tokens["labels"] = label_tokens["input_ids"]

    return input_tokens


In [19]:
# combine sentences in abstract and remove the <s> and </s> tokens at the beginning and end of the sentence
def abstract_combiner(abstract):
    combined = ""
    for sentence in abstract:
        combined += (sentence[4:-4]) + " "
    return combined
#abstract_combiner(rpapers["train"][6]["abstract_text"])

In [20]:
rpapers = papers_split
rpapers

DatasetDict({
    train: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'],
        num_rows: 5152
    })
    test: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'],
        num_rows: 1288
    })
})

In [ ]:
#x = mine[0]

In [24]:
tokenizer.pad_token = tokenizer.eos_token #we set th epadding to be true, it gives error wo this line
tokenized_papers = rpapers.map(preprocess_function, batched=True, batch_size=2)

Map:   0%|          | 0/5152 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD

Map:   0%|          | 0/1288 [00:00<?, ? examples/s]

Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token': '[PAD]'} not recognized.
Keyword arguments {'pad_token':

In [25]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [26]:
# ROUGE Score evaluation metric
import evaluate

rouge = evaluate.load("rouge")

In [28]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [30]:
!pip install accelerate -U

In [31]:
training_args = TrainingArguments(
    output_dir="./fine-tuned-gpt-neo",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_papers["train"],
    eval_dataset=tokenized_papers["test"],
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine-tuned-gpt-neo")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
eval_pred = trainer.evaluate()

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
compute_metrics(eval_pred)